In [15]:
import pandas as pd
import numpy as np
import os

In [16]:
# Sample data

DEFAULT_DATE_RANGE = ('2023-01-01', '2025-01-01')

def generate_timeseries_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS") # freq month start
    n_points = len(idx)
    rng = np.random.default_rng(np.random.randint(1, 99))
    base = rng.normal(loc=5500, scale=1500, size=n_points)
    trend = np.linspace(0, 2000, n_points)
    values = np.maximum(base + trend, 1000).astype(float)
    values = np.minimum(values, 10000).round(2)
    df = pd.DataFrame({"value": values})
    df.insert(0, "date", idx)
    return df
    
def generate_multiline_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS")

    rng = np.random.default_rng(np.random.randint(1, 99))
    series_names = ["North", "South", "East", "West"]
    frames: list[pd.DataFrame] = []
    for i, name in enumerate(series_names):
        values = np.maximum(
            rng.normal(loc=700_000 + i * 120_000, scale=250_000, size=len(idx))
            + np.linspace(0, 400_000, len(idx)),
            0,
        ).round(2)
        frames.append(pd.DataFrame({"date": idx, "series": name, "value": values}))

    df = pd.concat(frames, ignore_index=True)
    return df

In [17]:
bar = generate_timeseries_df()
single_line = generate_timeseries_df()
multi_line = generate_multiline_df()

# Bar chart

In [18]:
bar.head()

,date,value
0,2023-01-01,5038.56
1,2023-02-01,7607.72
2,2023-03-01,5955.91
3,2023-04-01,7859.84
4,2023-05-01,6076.64


# Single line chart

In [19]:
single_line.head()

,date,value
0,2023-01-01,4261.07
1,2023-02-01,5947.09
2,2023-03-01,5570.89
3,2023-04-01,5608.45
4,2023-05-01,6318.51


# Multi line chart

In [20]:
multi_line.head()

,date,series,value
0,2023-01-01,North,910565.33
1,2023-02-01,North,0.00
2,2023-03-01,North,657077.36
3,2023-04-01,North,1112471.98
4,2023-05-01,North,455676.30


In [21]:
# Export the final dfs needed for the charts
os.makedirs('output', exist_ok=True)
bar.to_csv('output/bar.csv', index=False)
single_line.to_csv('output/single_line.csv', index=False)
multi_line.to_csv('output/multi_line.csv', index=False)
